In [2]:
import requests
import json
import time
import csv
import pandas as pd
import numpy as np

In [12]:
import requests
import time
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

API_KEY   = "iYO3sditsNW8LVE8oua33BnEPGcKevHL5KqhO8sgoEFxKRD4yyV+oRZfDpGlKbuzhAqzvlZk5ja2qHXKpfnDCg=="
BASE_URL  = "http://apis.data.go.kr/B552584/EvCharger/getChargerInfo"
ROWS      = 1000
DATA_TYPE = "JSON"

# --- 세션에 백오프+재시도 설정을 붙여둡니다 ---
def create_retry_session(
    retries: int = 3,
    backoff_factor: float = 0.3,
    status_forcelist: tuple = (500, 502, 504),
):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

def fetch_all_charger_info(max_overall_retries: int = 2):
    """
    전체 호출에 실패하면 max_overall_retries 만큼 재시도합니다.
    페이지별로는 세션 설정에 따라 자동 재시도되고, 
    그럼에도 에러가 나면 스킵합니다.
    """
    attempt = 0
    while attempt <= max_overall_retries:
        try:
            session = create_retry_session()
            all_data = []

            # 1) 총 건수 가져오기
            params = {
                "serviceKey": API_KEY,
                "numOfRows": ROWS,
                "pageNo": 1,
                "dataType": DATA_TYPE
            }
            resp = session.get(BASE_URL, params=params, timeout=10)
            resp.raise_for_status()
            result = resp.json()
            total_count = int(result.get("totalCount", 0))
            total_pages = (total_count // ROWS) + (1 if total_count % ROWS else 0)
            print(f"총 충전기 수: {total_count}건 → {total_pages}페이지")

            # 2) 페이지별 수집
            for page in range(1, total_pages + 1):
                print(f"[{page}/{total_pages}] 수집 중…", end=" ")
                params["pageNo"] = page
                try:
                    r = session.get(BASE_URL, params=params, timeout=10)
                    r.raise_for_status()
                    data = r.json()
                    items = data.get("items", {}).get("item", [])
                    if isinstance(items, dict):
                        all_data.append(items)
                    elif isinstance(items, list):
                        all_data.extend(items)
                    else:
                        print("→ 데이터 없음")
                        continue
                    print("완료")
                except Exception as e:
                    # 마지막 시도에도 실패하면 스킵
                    print(f"⚠️ 페이지 {page} 오류: {e} → 스킵")
                time.sleep(0.2)

            return all_data

        except Exception as e:
            attempt += 1
            print(f"❌ 전체 수집 실패 (시도 {attempt}/{max_overall_retries}): {e}")
            if attempt > max_overall_retries:
                print("💥 최대 재시도 횟수 초과, 종료합니다.")
                return []
            print("🔄 1초 후 다시 시도합니다…")
            time.sleep(1)

if __name__ == "__main__":
    chargers = fetch_all_charger_info()
    if chargers:
        with open("./02_interim_data/all_ev_chargers.json", "w", encoding="utf-8") as f:
            json.dump(chargers, f, ensure_ascii=False, indent=2)
        print(f"\n✅ 총 {len(chargers)}건 저장 완료: all_ev_chargers.json")
    else:
        print("❌ 데이터를 가져오지 못했습니다.")


총 충전기 수: 445322건 → 446페이지
[1/446] 수집 중… 완료
[2/446] 수집 중… 완료
[3/446] 수집 중… 완료
[4/446] 수집 중… 완료
[5/446] 수집 중… 완료
[6/446] 수집 중… 완료
[7/446] 수집 중… 완료
[8/446] 수집 중… 완료
[9/446] 수집 중… 완료
[10/446] 수집 중… 완료
[11/446] 수집 중… 완료
[12/446] 수집 중… 완료
[13/446] 수집 중… 완료
[14/446] 수집 중… 완료
[15/446] 수집 중… ⚠️ 페이지 15 오류: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /B552584/EvCharger/getChargerInfo?serviceKey=iYO3sditsNW8LVE8oua33BnEPGcKevHL5KqhO8sgoEFxKRD4yyV%2BoRZfDpGlKbuzhAqzvlZk5ja2qHXKpfnDCg%3D%3D&numOfRows=1000&pageNo=15&dataType=JSON (Caused by ReadTimeoutError("HTTPConnectionPool(host='apis.data.go.kr', port=80): Read timed out. (read timeout=10)")) → 스킵
[16/446] 수집 중… 완료
[17/446] 수집 중… 완료
[18/446] 수집 중… 완료
[19/446] 수집 중… 완료
[20/446] 수집 중… 완료
[21/446] 수집 중… 완료
[22/446] 수집 중… 완료
[23/446] 수집 중… 완료
[24/446] 수집 중… 완료
[25/446] 수집 중… 완료
[26/446] 수집 중… 완료
[27/446] 수집 중… 완료
[28/446] 수집 중… 완료
[29/446] 수집 중… 완료
[30/446] 수집 중… 완료
[31/446] 수집 중… 완료
[32/446] 수집 중… 완료
[33/446] 

In [ ]:


def json_to_csv(json_filepath="./02_interim_data/all_ev_chargers.json", csv_filepath="./02_interim_data/all_ev_chargers.csv"):
    # 1) JSON 파일 불러오기
    with open(json_filepath, "r", encoding="utf-8") as f:
        data = json.load(f)  # 리스트 of dict

    if not data:
        print("❌ 변환할 데이터가 없습니다.")
        return

    # 2) CSV 헤더(컬럼) 추출 — 모든 키들의 합집합
    #    (일부 항목이 빠진 충전소도 있어서, union 방식으로 헤더 생성)
    keys = set()
    for entry in data:
        keys.update(entry.keys())
    header = list(keys)

    # 3) CSV 파일 쓰기
    with open(csv_filepath, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        for entry in data:
            writer.writerow(entry)

    print(f"✅ CSV 변환 완료: {csv_filepath}")

if __name__ == "__main__":
    json_to_csv()

In [17]:
import pandas as pd

df = pd.read_csv('./02_interim_data/all_ev_chargers.csv', dtype={'year': 'Int64', 'zcode': 'Int64'})

zcode_abbr = {
    11: '서울', 26: '부산', 27: '대구', 28: '인천', 29: '광주',
    30: '대전', 31: '울산', 36: '세종', 41: '경기', 51: '강원',
    43: '충북', 44: '충남', 52: '전북', 46: '전남', 47: '경북',
    48: '경남', 50: '제주'
}
df['sido'] = (
    df['zcode']
    .astype(str)          # 문자열로 변환
    .str[:2]              # 앞 두 글자만
    .astype(int)          # 정수로
    .map(zcode_abbr)      # 딕셔너리로 매핑
)

slow_types = {'02', '07', '08'}
df['charge_speed'] = df['chgerType'].astype(str).str.zfill(2).apply(
    lambda x: '완속' if x in slow_types else '급속'
)
region_yearly = (
    df.dropna(subset=['year', 'sido'])
      .groupby(['sido', 'year', 'charge_speed'])
      .size()
      .reset_index(name='count')
)

national = (
    region_yearly
    .groupby(['year', 'charge_speed'])['count']
    .sum()
    .reset_index()
    .assign(sido='전국')
)

combined = pd.concat([region_yearly, national], ignore_index=True)

combined = combined.sort_values(['sido', 'charge_speed', 'year'])
combined['cumulative_count'] = (
    combined
    .groupby(['sido', 'charge_speed'])['count']
    .cumsum()
)

pivot_df = (
    combined
    .pivot_table(
        index=['sido', 'year'],
        columns='charge_speed',
        values=['count', 'cumulative_count'],
        fill_value=0
    )
)

pivot_df.columns = [f"{speed}_{metric}" for metric, speed in pivot_df.columns]
pivot_df = pivot_df.reset_index()

pivot_df = pivot_df[[
    'sido', 'year',
    '급속_count', '급속_cumulative_count',
    '완속_count', '완속_cumulative_count'
]]

pivot_df.columns = [
    '시도', '연도',
    '급속 설치건수', '급속 누적건수',
    '완속 설치건수', '완속 누적건수'
]

pivot_df.to_csv('./02_interim_data/ev_charger_yearly_pivot.csv', index=False, encoding='utf-8-sig')

C:\Users\junjo\AppData\Local\Temp\ipykernel_67372\3406886271.py:3: DtypeWarning: Columns (5,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./02_interim_data/all_ev_chargers.csv', dtype={'year': 'Int64', 'zcode': 'Int64'})


In [18]:

# 연월 datetime 추가 (1월 기준)
pivot_df['연월'] = pd.to_datetime(pivot_df['연도'].astype(str) + '-01')

# 월 전체 범위 만들기
full_months = pd.date_range(
    start=pivot_df['연월'].min(),
    end=pivot_df['연월'].max() + pd.offsets.YearEnd(0),
    freq='MS'
)

# 시도별로 월 보간
interpolated_dfs = []

for sido, group in pivot_df.groupby('시도'):
    group = group.set_index('연월')

    # 연도만 있을 테니, 월 보간을 위해 전체 월 인덱스 추가
    group = group.reindex(full_months)
    group['시도'] = sido
    group.index.name = '연월'

    # 누적 건수 보간
    for col in ['급속 누적건수', '완속 누적건수']:
        group[col] = group[col].interpolate(method='linear', limit_direction='both')

    # 월별 설치 건수 = 누적건수.diff()
    group['급속 설치건수'] = group['급속 누적건수'].diff().fillna(group['급속 누적건수'])
    group['완속 설치건수'] = group['완속 누적건수'].diff().fillna(group['완속 누적건수'])

    # 정수로 변환
    for col in ['급속 설치건수', '급속 누적건수', '완속 설치건수', '완속 누적건수']:
        group[col] = group[col].round().astype('Int64')

    interpolated_dfs.append(group.reset_index())

# 합치기
monthly_df = pd.concat(interpolated_dfs, ignore_index=True)

# 연월 문자열로 변환
monthly_df['연월'] = monthly_df['연월'].dt.strftime('%Y-%m')

# 정렬
monthly_df = monthly_df.sort_values(['시도', '연월']).drop('연도', axis=1)

# 저장
monthly_df.to_csv('./02_interim_data/ev_charger_monthly_interpolated.csv', index=False, encoding='utf-8-sig')
